In [1]:
import numpy as np
import pandas as pd

import csv

# Open the TSV file in read mode with 't' flag for text mode
with open('/Users/lujun.li/projects/causallm-github/causalllm/resource/input/machine_translation/floresp-v2.0-rc.2/metadata_devtest.tsv', 'rt') as tsvfile:
    # Create a CSV reader object with tab delimiter
    reader = csv.reader(tsvfile, delimiter='\t')
    
    # Iterate through each row in the TSV file
    for row in reader:
        # Do something with each row, for example, print it
        print(row)



['URL', 'domain', 'topic', 'has_image', 'has_hyperlink']
['https://en.wikinews.org/wiki/Toronto_team-led_research_on_Type_1_Diabetes_%27groundbreaking%27', 'wikinews', 'disease, research, canada', 'no', 'yes']
['https://en.wikinews.org/wiki/Toronto_team-led_research_on_Type_1_Diabetes_%27groundbreaking%27', 'wikinews', 'disease, research, canada', 'no', 'yes']
['https://en.wikinews.org/wiki/Toronto_team-led_research_on_Type_1_Diabetes_%27groundbreaking%27', 'wikinews', 'disease, research, canada', 'no', 'yes']
['https://en.wikinews.org/wiki/Nobel_Prize_in_Literature_Committee_abandons_efforts_to_contact_Bob_Dylan', 'wikinews', 'music', 'yes', 'yes']
['https://en.wikinews.org/wiki/Nobel_Prize_in_Literature_Committee_abandons_efforts_to_contact_Bob_Dylan', 'wikinews', 'music', 'yes', 'yes']
['https://en.wikinews.org/wiki/Amazon_to_buy_smart_doorbell_startup_Ring', 'wikinews', 'business', 'no', 'yes']
['https://en.wikinews.org/wiki/Amazon_to_buy_smart_doorbell_startup_Ring', 'wikinews', '

In [6]:
from datasets import load_dataset
datasets = {}

configs = ["eng_Latn", "mlt_Latn", "ind_Latn", "lvs_Latn", "isl_Latn", "khm_Khmr"]

code_to_language_dict = {
    "eng_Latn": "english",
    "mlt_Latn": "maltese",
    "ind_Latn": "indonesian",
    "lvs_Latn": "standard latvian",
    "isl_Latn": "icelandic",
    "khm_Khmr": "khmer"
}

for config in configs:
    datasets[config] = load_dataset("facebook/flores", config)

for config, dataset in datasets.items():
    dataset.save_to_disk(f"resource/input/datasets/{config}")

Generating dev split: 997 examples [00:00, 12690.26 examples/s]
Generating devtest split: 1012 examples [00:00, 35495.01 examples/s]
Generating dev split: 997 examples [00:00, 30922.57 examples/s]
Generating devtest split: 1012 examples [00:00, 29377.69 examples/s]
Generating dev split: 997 examples [00:00, 30874.63 examples/s]
Generating devtest split: 1012 examples [00:00, 29093.77 examples/s]
Generating dev split: 997 examples [00:00, 32688.59 examples/s]
Generating devtest split: 1012 examples [00:00, 32943.74 examples/s]
Generating dev split: 997 examples [00:00, 28414.03 examples/s]
Generating devtest split: 1012 examples [00:00, 30066.27 examples/s]
Saving the dataset (1/1 shards): 100%|██████████| 1012/1012 [00:00<00:00, 229229.12 examples/s]


In [20]:
english_ds = load_dataset(f"resource/input/datasets/eng_Latn", split="validation")
mlt_ds = load_dataset(f"resource/input/datasets/mlt_Latn", split="validation")

In [32]:
dataframes = {}
for dataset in datasets:
    df = pd.DataFrame(datasets[dataset]["devtest"])
    dataframes[dataset] = df
    dataframes[dataset].to_parquet(f"resource/input/datasets/{dataset}.parquet", index=False)

DatasetDict({
    dev: Dataset({
        features: ['id', 'URL', 'domain', 'topic', 'has_image', 'has_hyperlink', 'sentence'],
        num_rows: 997
    })
    devtest: Dataset({
        features: ['id', 'URL', 'domain', 'topic', 'has_image', 'has_hyperlink', 'sentence'],
        num_rows: 1012
    })
})
DatasetDict({
    dev: Dataset({
        features: ['id', 'URL', 'domain', 'topic', 'has_image', 'has_hyperlink', 'sentence'],
        num_rows: 997
    })
    devtest: Dataset({
        features: ['id', 'URL', 'domain', 'topic', 'has_image', 'has_hyperlink', 'sentence'],
        num_rows: 1012
    })
})
DatasetDict({
    dev: Dataset({
        features: ['id', 'URL', 'domain', 'topic', 'has_image', 'has_hyperlink', 'sentence'],
        num_rows: 997
    })
    devtest: Dataset({
        features: ['id', 'URL', 'domain', 'topic', 'has_image', 'has_hyperlink', 'sentence'],
        num_rows: 1012
    })
})
DatasetDict({
    dev: Dataset({
        features: ['id', 'URL', 'domain', 'topic'

In [35]:
eng_ds = pd.read_parquet("/Users/lujun.li/projects/causallm-github/causalllm/resource/input/datasets/eng_Latn.parquet")

In [36]:
mlt_Latn_ds = pd.read_parquet("/Users/lujun.li/projects/causallm-github/causalllm/resource/input/datasets/mlt_Latn.parquet")

In [54]:
merged_df_list = []
eng_ds =  pd.read_parquet("/Users/lujun.li/projects/causallm-github/causalllm/resource/input/datasets/eng_Latn.parquet")
for dataset_name in ["mlt_Latn", "ind_Latn", "lvs_Latn", "isl_Latn", "khm_Khmr"]:
    dataset_target = pd.read_parquet(f"/Users/lujun.li/projects/causallm-github/causalllm/resource/input/datasets/{dataset_name}.parquet")
    merged_df = pd.merge(eng_ds, dataset_target, on="id", how="inner",suffixes=("_eng", f"_target"))[["id",f"sentence_target","sentence_eng"]]
    merged_df["dataset_name"] = dataset_name
    merged_df_list.append(merged_df)
concatenated_df = pd.concat(merged_df_list, ignore_index=True)

In [57]:
concatenated_df

,id,sentence_target,sentence_eng,dataset_name
0,1,"""Issa għandna ġrieden ta' 4 xhur li mhumiex di...","""We now have 4-month-old mice that are non-dia...",mlt_Latn
1,2,"Dr. Ehud Ur, professur tal-mediċina fl-Univers...","Dr. Ehud Ur, professor of medicine at Dalhousi...",mlt_Latn
2,3,"Bħal xi esperti oħra, huwa xettiku dwar jekk i...","Like some other experts, he is skeptical about...",mlt_Latn
3,4,"Nhar it-Tnejn, Sara Danius, segretarja permane...","On Monday, Sara Danius, permanent secretary of...",mlt_Latn
4,5,"Danius qal, ""Bħalissa m’aħna qed nagħmlu xejn....","Danius said, ""Right now we are doing nothing. ...",mlt_Latn
...,...,...,...,...
5055,1008,ដោយសារតំបន់នេះមានប្រជាជនរប៉េះរប៉ោះ អាស្រ័យហេតុ...,"As the areas are sparsely populated, and light...",khm_Khmr
5056,1009,វប្បធម៌នៃការធ្វើការរបស់ជនជាតិជប៉ុន គឺមានលក្ខណៈ...,Japanese work culture is more hierarchical and...,khm_Khmr
5057,1010,ឈុតខោអាវធំគឺជាសម្លៀកបំពាក់ស្តង់ដារសម្រាប់អ្នកជ...,"Suits are standard business attire, and cowork...",khm_Khmr
5058,1011,ភាពចុះសម្រុងគ្នានៅកន្លែងធ្វើការគឺមានសារៈសំខាន់...,"Workplace harmony is crucial, emphasizing grou...",khm_Khmr


In [59]:
arraycounter = 0
system_message = """"
You are a translation expert, please help me translate the sentence I give to you. Don't answer irrelevant words such as "of course", just reply to me the translated sentence directly.
"""
initial_prompt = """
Instruction: {system_message}
Input: 
{INSTRUCTION}
{INPUT}
"""
mixtral_instruct_initial_prompt = """<s>[INST]
{system_message}
{INSTRUCTION}
{INPUT}
[/INST] 
</s>"""

llama2_chat_initial_prompt = """
<s>[INST] <<SYS>>
{system_message}
<</SYS>>
{INSTRUCTION}
{INPUT}
[/INST]
"""

llama3_chat_init_prompt = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
{system_message}
<|eot_id|><|start_header_id|>user<|end_header_id|>
{INSTRUCTION}
{INPUT}
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

code_to_language_dict = {
    "eng_Latn": "English",
    "mlt_Latn": "Maltese",
    "ind_Latn": "Indonesian",
    "lvs_Latn": "Standard Latvian",
    "isl_Latn": "Icelandic",
    "khm_Khmr": "Khmer"
}


In [60]:
from langchain import PromptTemplate

initial_prompt_template = PromptTemplate.from_template(initial_prompt)
llama2_chat_initial_prompt_template = PromptTemplate.from_template(
    llama2_chat_initial_prompt
)
llama3_chat_initial_prompt_template = PromptTemplate.from_template(
    llama3_chat_init_prompt
)
mixtral_instruct_initial_prompt_template = PromptTemplate.from_template(
    mixtral_instruct_initial_prompt
)


def generate_initial_prompts(row):
    language_code = row["dataset_name"]
    language_name = code_to_language_dict[language_code]
    INSTRUCTION = f"Please translate the following English sentence to {language_name}."
    INPUT = row["sentence_eng"]
    initial_prompt = initial_prompt_template.format(INSTRUCTION=INSTRUCTION, system_message=system_message, INPUT=INPUT)
    llama2_chat_initial_prompt = llama2_chat_initial_prompt_template.format(
        INSTRUCTION=INSTRUCTION, system_message=system_message, INPUT=INPUT
    )
    llama3_chat_initial_prompt = llama3_chat_initial_prompt_template.format(
        INSTRUCTION=INSTRUCTION, system_message=system_message, INPUT=INPUT
    )
    mixtral_instruct_initial_prompt = mixtral_instruct_initial_prompt_template.format(
        INSTRUCTION=INSTRUCTION, system_message=system_message, INPUT=INPUT
    )
    row["initial_prompt"] = initial_prompt
    row["llama2_chat_initial_prompt"] = llama2_chat_initial_prompt
    row["llama3_chat_initial_prompt"] = llama3_chat_initial_prompt
    row["mixtral_instruct_initial_prompt"] = mixtral_instruct_initial_prompt
    row["SYSTEM_MESSAGE"] = system_message
    row["USER_MESSAGE"] = INPUT
    return row


concatenated_df = concatenated_df.apply(
    generate_initial_prompts, axis=1
)

In [69]:
len(concatenated_df)

5060

In [70]:
concatenated_df.to_csv("resource/input/machine_translation/ml_experiment_prompt.csv")

In [72]:
concatenated_df["dataset_name"].value_counts()

dataset_name
mlt_Latn    1012
ind_Latn    1012
lvs_Latn    1012
isl_Latn    1012
khm_Khmr    1012
Name: count, dtype: int64